In [1]:
import openpyxl
import mne
import mnelab
import numpy
import matplotlib
import xgboost
import pyriemann
import scipy
import pickle
import numpy as np
import pandas as pd
import FeatureExtract
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
# 行列标识符,P300二分类转字符36分类用
chars = {'A': (1, 7), 'B': (1, 8), 'C': (1, 9), 'D': (1, 10), 'E': (1, 11), 'F': (1, 12), 
         'G': (2, 7), 'H': (2, 8), 'I': (2, 9), 'J': (2, 10), 'K': (2, 11), 'L': (2, 12), 
         'M': (3, 7), 'N': (3, 8), 'O': (3, 9), 'P': (3, 10), 'Q': (3, 11), 'R': (3, 12), 
         'S': (4, 7), 'T': (4, 8), 'U': (4, 9), 'V': (4, 10), 'W': (4, 11), 'X': (4, 12), 
         'Y': (5, 7), 'Z': (5, 8), '1': (5, 9), '2': (5, 10), '3': (5, 11), '4': (5, 12), 
         '5': (6, 7), '6': (6, 8), '7': (6, 9), '8': (6, 10), '9': (6, 11), '0': (6, 12), 
        }

In [19]:
# 测试数据加载
raw_test_data = openpyxl.load_workbook(u'/home/chris/predict_code/lyft/bci/data/S5/S5_test_data.xlsx')
raw_test_event = openpyxl.load_workbook(u'/home/chris/predict_code/lyft/bci/data/S5/S5_test_event.xlsx')

# 测试数据预处理，特征提取, 10*5*12*521
# (即10个字符，每个五轮，每轮12次闪烁，20个通道，每个通道26个特征，共520特征,外加一个字符矩阵行列标识符)
test_data = []

# 带通滤波器上下界
freq_min, freq_max = 1, 30
# 调用8阶带通滤波器
# b, a = scipy.signal.butter(8, [0.02,0.8], 'bandpass')

for sheet in raw_test_data.sheetnames:
    # 取sheet数据
    table_event = raw_test_event[sheet]
    table_data = raw_test_data[sheet]
    
    # 转换后数据,5轮次P300脑电处理后数据
    iter = 0 # 轮次数
    count = 0 # 计数
    char_data = [] # 多轮数据, 5*12*521
    iter_data = [] # 一轮数据, 12*521
    for row in range(raw_test_event[sheet].max_row):
        if table_event.cell(row+1, 1).value >= 1 and table_event.cell(row+1, 1).value <= 12:
            # data.shape: 275*20,-100ms~1000ms时间窗口
            # feature: 520
            feature = []
            data = []
            
            if count % 12 == 0:
                iter += 1
                count = 0
                char_data.append(iter_data)
                iter_data.clear()
            
            for i in range(20):
                channel = []
                for j in range(275):
                    channel.append(table_data.cell(table_event.cell(row+1, 2).value-24+j, i+1).value)
                data.append(channel)
            for i in range(20):
                # 带通滤波
                # filter_data = scipy.signal.filtfilt(b, a, data[i])
                filter_data = data[i]
                # 16个时域特征
                feature.append(FeatureExtract.mean_fea(filter_data))
                feature.append(FeatureExtract.rms_fea(filter_data))
                feature.append(FeatureExtract.sr_fea(filter_data))
                feature.append(FeatureExtract.am_fea(filter_data))
                feature.append(FeatureExtract.skew_fea(filter_data))
                feature.append(FeatureExtract.kurt_fea(filter_data))
                feature.append(FeatureExtract.max_fea(filter_data))
                feature.append(FeatureExtract.min_fea(filter_data))
                feature.append(FeatureExtract.pp_fea(filter_data))
                feature.append(FeatureExtract.var_fea(filter_data))
                feature.append(FeatureExtract.waveform_index(filter_data))
                feature.append(FeatureExtract.peak_index(filter_data))
                feature.append(FeatureExtract.impluse_factor(filter_data))
                feature.append(FeatureExtract.tolerance_index(filter_data))
                feature.append(FeatureExtract.skew_index(filter_data))
                feature.append(FeatureExtract.kurt_index(filter_data))
                # 10个频域特征
                feature.append(FeatureExtract.fft_mean(filter_data))
                feature.append(FeatureExtract.fft_var(filter_data))
                feature.append(FeatureExtract.fft_std(filter_data))
                feature.append(FeatureExtract.fft_entropy(filter_data))
                feature.append(FeatureExtract.fft_energy(filter_data))
                feature.append(FeatureExtract.fft_skew(filter_data))
                feature.append(FeatureExtract.fft_shape_mean(filter_data))
                feature.append(FeatureExtract.fft_shape_std(filter_data))
                feature.append(FeatureExtract.fft_shape_skew(filter_data))
                feature.append(FeatureExtract.fft_shape_kurt(filter_data))
                # 字符矩阵行列标识符
            feature.append(table_event.cell(row+1, 1).value)
            iter_data.append(feature)
            count += 1
    test_data.append(char_data)
            
print((np.array(test_data)).shape)    

(10, 5, 12, 521)


In [21]:
# P300二分类器模型加载,输出字符
answer = []
with open('model/xgboost1.pickle', 'rb') as fr:
    clf_p300 = pickle.load(fr)
    # 轮询每个字符数据
    for i in range(len(test_data)):
        # size:5, 12, 521
        current_char_data = np.array(test_data[i])
        char_prob = []
        for j in range(len(current_char_data)):
            # size: 12, 521
            curr_iter_data = current_char_data[j]
            # 12次闪烁后脑电含P300波的概率， 12×(prob, 对应row或colum号)
            p300_prob = []
            for k in range(len(curr_iter_data)):
                prob_pred = (np.squeeze(clf_p300.predict_proba(np.expand_dims(curr_iter_data[k][0:-2], 0))))[0]
                # 行列标识符和对应该行列闪烁后有无P300的概率组成元组
                p300_prob.append((prob_pred, curr_iter_data[k][-1]))
            char_prob.append(p300_prob)
        # print((np.array(char_prob)))
        # P300二分类检测完成，求字符
        prob = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] # 初始各行或列闪烁后有无P300概率
        for a in range(len(char_prob)):
            p300_iter = char_prob[a]
            # 多轮加权
            for b in range(len(p300_iter)):
                prob[int(p300_iter[b][1]-1)] += p300_iter[b][0]
            for index in range(len(prob)):
                prob[index] = prob[index]/5.0
        # print(prob)
        # 求出对应行列标识符
        char_row = np.argmax(prob[0:6])
        char_col = np.argmax(prob[6:])
        # 基于行列标识符，按值找键查字符矩阵标识符字典
        answer.append([k for k,v in chars.items() if v==(char_row+1, char_col+7)])
        print((char_row+1, char_col+7), answer[i])

(6, 12) ['0']
(3, 9) ['O']
(1, 11) ['E']
(4, 10) ['V']
(6, 8) ['6']
(2, 9) ['I']
(2, 9) ['I']
(3, 11) ['Q']
(4, 8) ['T']
(4, 9) ['U']
